The file contains the edges of a directed graph. Vertices are labeled as positive integers from 1 to 875714. Every row indicates an edge, the vertex label in first column is the tail and the vertex label in second column is the head (recall the graph is directed, and the edges are directed from the first column vertex to the second column vertex). So for example, the $11^{th}$ row looks like this : "2 47646". This just means that the vertex with label 2 has an outgoing edge to the vertex with label 47646.

Your task is to code up the algorithm from the video lectures for computing strongly connected components (SCCs), and to run this algorithm on the given graph.

Output Format: You should output the sizes of the 5 largest SCCs in the given graph, in decreasing order of sizes, separated by commas (avoid any spaces). So if your algorithm computes the sizes of the five largest SCCs to be 500, 400, 300, 200 and 100, then your answer should be "500,400,300,200,100" (without the quotes). If your algorithm finds less than 5 SCCs, then write 0 for the remaining terms. Thus, if your algorithm computes only 3 SCCs whose sizes are 400, 300, and 100, then your answer should be "400,300,100,0,0" (without the quotes). (Note also that your answer should not have any spaces in it.)

WARNING: This is the most challenging programming assignment of the course. Because of the size of the graph you may have to manage memory carefully. The best way to do this depends on your programming language and environment, and we strongly suggest that you exchange tips for doing this on the discussion forums.

In [7]:
import multiprocessing
class SCC(multiprocessing.Process):
    
    def __init__(self, file, map_len):
        self.placenum = 0
        self.map_len = map_len
        self.vertnum = map_len
        self.explored = [False for i in range(1, map_len+1)]  
        self.explored2 = [False for i in range(1, map_len+1)]  
        self.visit_order = [0 for i in range(1, map_len+1)]  
        self.sccs = [0 for i in range(1, map_len+1)]
        self.G = {i : [] for i in range(1, map_len+1)}
        self.G_rev = {i : [] for i in range(1, map_len+1)}        
        with open(file) as f:
            for line in f:
                lst = [int(num) for num in line.split()]
                self.G[lst[0]].append(lst[1])
                self.G_rev[lst[1]].append(lst[0])             
        
    def dfs1(self, vert_idx):  
        for idx in self.G_rev[vert_idx]:  
            if self.explored[idx-1] == False:  
                    self.explored[idx-1] = True  
                    self.dfs1(idx)
        self.vertnum -= 1                
        self.visit_order[self.vertnum] = vert_idx   
        
    def dfs2(self, vert_idx):
        for idx in self.G[vert_idx]:  
            if self.explored2[idx-1] == False:  
                    self.explored2[idx-1] = True  
                    self.dfs2(idx)  
        self.sccs[self.placenum-1] += 1
    
    def dfs_loop(self):
        for idx in range(1, self.map_len+1):  
            if self.explored[idx-1] == False:  
                    self.explored[idx-1] = True  
                    self.dfs1(idx)  
       
        for idx in self.visit_order:  
            if self.explored2[idx-1] == False:  
                    self.placenum = idx  
                    self.explored2[idx-1] = True  
                    self.dfs2(idx)  

        return sorted(self.sccs, reverse = True)
    
    def answer(self, outnum):
        ans = self.dfs_loop()
        ans = ans[:outnum]
        return ', '.join(str(num) for num in ans)

In [8]:
from sys import setrecursionlimit
from threading import stack_size, Thread
myfile = 'SCC.txt'
vertnum = 875714
output_num = 5
setrecursionlimit(100000)
stack_size(65000000)

65000000

In [9]:
%%time
scc = SCC(myfile, vertnum)
thread = Thread(target = scc.answer, args = (output_num,))
thread.start()
thread.join()
ans = scc.answer(output_num)
print("The answer is {}.".format(ans))

The answer is 434821, 968, 459, 313, 211.
Wall time: 10.6 s
